# Evaluate scheduled visits

In [2]:
import os
import numpy as np
import pandas as pd
import scipy as sp
import sys
sys.path.append('/home/s/seanmacb/LSST-Camera/Extras and Utilities/pkgs')
sys.path.append('/home/s/seanmacb/LSST-Camera/Extras and Utilities/pkgs/eo_pipe/python/lsst/eo/pipe')
# import eo_pipe.python.lsst.eo.pipe as eo_pipe
# from eo_pipe.python.lsst.eo.pipe import (readNoiseTask, darkCurrentTask, defectsTask, eperTask, divisaderoTearingTask, ptcPlotsTask,linearityPlotsTask, bfAnalysisTask)
import copy
import pickle
import lsst.daf.butler as daf_butler
import lsst.afw.math as afw_math
from matplotlib import pyplot as plt
from lsst.ip.isr import IsrTask, IsrTaskConfig
plt.style.use("seaborn-v0_8-talk")
from matplotlib import lines
from mpl_toolkits import axes_grid1
from astropy.stats import sigma_clip
from scipy.stats import skew
import astropy
from tabulate import tabulate
from scipy.stats import norm
import rubin_scheduler

from lsst.obs.lsst import LsstCam, LsstTS8
from IPython.display import display_html, HTML
# import myutils_ar.myutils as myu

from astropy.time import Time
from collections import defaultdict
from datetime import datetime, timedelta
from scipy.stats import norm

# LSST Specific Libraries
from lsst_efd_client import EfdClient
from lsst.summit.utils.plotting import plot
from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.tmaUtils import (
    getCommandsDuringEvent,
    TMAEvent,
    TMAEventMaker,
    TMAState,
)
import matplotlib.dates as mdates

async def query_bump_logs_in_chunks(
    start_date,
    end_date,
    client_name="",
    chunk_size_days=3,
    topic_name="lsst.sal.MTM1M3.logevent_logMessage",
    fields=["message"],
):
    """
    Queries the log messages related to bump tests from the EFD in chunks.

    Args:
        start_date (str): Start date of the query in ISO format (YYYY-MM-DD).

        end_date (str): End date of the query in ISO format (YYYY-MM-DD).

        client_name (str, optional): Name of the EFD client. Defaults to "".

        chunk_size_days (int, optional): Number of days per chunk. Defaults to 3.

        topic_name (str, optional): SAL topic name to be queried by the client. Defaults to lsst.sal.MTM1M3.logevent_logMessage.

        fields (list[str], optional): Fields to be queried by the client. Defaults to ["message"].

    Returns:
        pandas.DataFrame: Concatenated DataFrame containing the queried log messages.
    """

    client = makeClient(client_name)

    # Convert start and end dates to datetime objects
    start = datetime.fromisoformat(start_date)
    end = datetime.fromisoformat(end_date)

    # Initialize an empty DataFrame to store concatenated results
    all_data = pd.DataFrame()

    current_start = start
    while current_start < end:
        current_end = min(current_start + timedelta(days=chunk_size_days), end)
        try:
            # Query the data for the current chunk
            chunk_data = await client.select_time_series(
                topic_name=topic_name,
                fields=fields,
                start=Time(current_start.isoformat(), format="isot", scale="utc"),
                end=Time(current_end.isoformat(), format="isot", scale="utc"),
            )
            # Concatenate the chunk data to the main DataFrame
            all_data = pd.concat([all_data, chunk_data], ignore_index=False)
        except Exception as e:
            print(
                f"Error querying data from {current_start.isoformat()} to {current_end.isoformat()}: {e}"
            )
            continue  # Optionally, continue to the next chunk

        # Move to the next chunk
        current_start = current_end

    return all_data

from astropy.utils.iers import conf
conf.auto_max_age = None

def makeClient(client_name):
    # Create the client based on client_name
    if client_name == "summit_efd":
        return makeEfdClient("summit_efd")
    elif client_name == "usdf_efd":
        return makeEfdClient("usdf_efd")
    elif client_name == "idf_efd":
        return makeEfdClient("idf_efd")
    else:
        return makeEfdClient()  # Default client


# Example usage:
# begin = "2023-11-13T01:00"
# end = "2023-12-21T01:00"
# bump_logs = await query_bump_logs_in_chunks(begin, end, client_name='')

import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib') 
def showAndClear():
    plt.show()
    # Clear the current axes.
    plt.cla()
    # Clear the current figure.
    plt.clf()
    # Closes all the figure windows.
    plt.close("all")
    plt.close(fig)

    return


async def getDataFrame(client, starts, ends, topic, verbose=True, fields=None):

    all_data = pd.DataFrame()
    for start, end in zip(starts, ends):
        if verbose:
            print(
                r"Starting query for time range {} - {}".format(start, end),
                end=" . . . ",
            )
        if fields != None:
            df_bump = await client.select_time_series(
                topic, fields, Time(start), Time(end)
            )
        else:
            df_bump = await client.select_time_series(
                topic, "*", Time(start), Time(end)
            )

        all_data = pd.concat([all_data, df_bump], ignore_index=False)

        del df_bump

        if verbose:
            print("Finished")

    return all_data


def makeDateRange(startPoint, endPoint, step=np.timedelta64(1, "D")):
    starts = np.arange(startPoint, endPoint, step=step)
    ends = starts + np.timedelta64(1, "D")
    return starts, ends

def fitGaussian(data, ax):
    mu, std = norm.fit(data)

    xmin, xmax = ax.get_xlim()
    x = np.linspace(np.floor(xmin), np.ceil(xmax), int(10e4))
    p = norm.pdf(x, mu, std)

    return mu, std, p, x, xmin, xmax

def getFWHM_from_gaussian(sigma):
    return 2 * np.sqrt(np.log(2) * 2) * sigma

from astropy.time import Time
from astropy.coordinates import get_sun
import astropy.coordinates as coord
import astropy.units as u



from astropy.stats import mad_std
# instrument="LSSTCam"
# embargoButler = daf_butler.Butler("/repo/embargo",collections=['LSSTCam/raw/all','LSSTCam/calib'])
# mainButler = daf_butler.Butler("/repo/main",collections=['LSSTCam/raw/all','LSSTCam/calib/unbounded'])
# flatColl = "LSSTCam/calib/DM-50448/initial-ugr-flats/flatGen-u.20250423a"
# # flat_det184 = embargoButler.get("flat",collections=flatColl,detector=184,instrument=instrument,physical_filter="i_39")
# isrConfig = IsrTaskConfig()
# isrConfig.doLinearize=False
# isrConfig.doOverscan=True
# isrConfig.overscan.fitType="MEDIAN_PER_ROW"
# isrConfig.overscan.doParallelOverscan=True
# isrConfig.doAssembleCcd=True
# isrConfig.doBias=False
# isrConfig.doVariance=False
# isrConfig.doCrosstalk=False
# isrConfig.doBrighterFatter=False
# isrConfig.doDark=False
# isrConfig.doStrayLight=False
# isrConfig.doFlat=False
# isrConfig.doFringe=False
# isrConfig.doApplyGains=True
# isrConfig.usePtcGains=False
# isrConfig.doDefect=False
# isrConfig.doNanMasking=True
# isrConfig.doInterpolate=False
# isrConfig.doSaturation=False
# isrConfig.doSaturationInterpolation=False
# isrTask = IsrTask(config=isrConfig)
async def query_bump_logs_in_chunks(
    start_date, end_date, client_name="", chunk_size_days=3,topic_name="lsst.sal.MTM1M3.logevent_logMessage",fields=["message"]
):
    """
    Queries the log messages related to bump tests from the EFD in chunks.

    Args:
        start_date (str): Start date of the query in ISO format (YYYY-MM-DD).
        
        end_date (str): End date of the query in ISO format (YYYY-MM-DD).
        
        client_name (str, optional): Name of the EFD client. Defaults to "".
        
        chunk_size_days (int, optional): Number of days per chunk. Defaults to 3.

        topic_name (str, optional): SAL topic name to be queried by the client. Defaults to lsst.sal.MTM1M3.logevent_logMessage.

        fields (list[str], optional): Fields to be queried by the client. Defaults to ["message"].

    Returns:
        pandas.DataFrame: Concatenated DataFrame containing the queried log messages.
    """

    client = makeClient(client_name)

    # Convert start and end dates to datetime objects
    start = datetime.fromisoformat(start_date)
    end = datetime.fromisoformat(end_date)

    # Initialize an empty DataFrame to store concatenated results
    all_data = pd.DataFrame()

    current_start = start
    while current_start < end:
        current_end = min(current_start + timedelta(days=chunk_size_days), end)
        try:
            # Query the data for the current chunk
            chunk_data = await client.select_time_series(
                topic_name=topic_name,
                fields=fields,
                start=Time(current_start.isoformat(), format="isot", scale="utc"),
                end=Time(current_end.isoformat(), format="isot", scale="utc"),
            )
            # Concatenate the chunk data to the main DataFrame
            all_data = pd.concat([all_data, chunk_data], ignore_index=False)
        except Exception as e:
            print(
                f"Error querying data from {current_start.isoformat()} to {current_end.isoformat()}: {e}"
            )
            continue  # Optionally, continue to the next chunk

        # Move to the next chunk
        current_start = current_end

    return all_data

def makeClient(client_name):
        # Create the client based on client_name
    if client_name == "summit_efd":
        return makeEfdClient("summit_efd")
    elif client_name == "usdf_efd":
        return makeEfdClient("usdf_efd")
    elif client_name == "idf_efd":
        return makeEfdClient("idf_efd")
    elif client_name == "base_efd":
        return makeEfdClient("base_efd")
    else:
        return makeEfdClient()  # Default client


# Example usage:
# begin = "2023-11-13T01:00"
# end = "2023-12-21T01:00"
# bump_logs = await query_bump_logs_in_chunks(begin, end, client_name='')

def showAndClear():
    plt.show()
    # Clear the current axes.
    plt.cla() 
    # Clear the current figure.
    plt.clf() 
    # Closes all the figure windows.
    plt.close('all')   
    plt.close(fig)
    
    return

import io
import os
import math
import pickle
import importlib

import urllib.request

import numpy as np
import healpy as hp


async def getDataFrame(client,starts,ends,topic,verbose=True,fields=None):
    
    all_data = pd.DataFrame()
    for start,end in zip(starts,ends):
        if verbose:
            print(r"Starting query for time range {} - {}".format(start,end),end=" . . . ")
        if fields != None:
            df_bump = await client.select_time_series(topic,fields, Time(start), Time(end))
        else:
            df_bump = await client.select_time_series(topic,"*", Time(start), Time(end))
        
        all_data = pd.concat([all_data, df_bump], ignore_index=False)

        del df_bump

        if verbose:
            print("Finished")
    
    return all_data

import requests

def makeDateRange(startPoint,endPoint,step=np.timedelta64(1, 'D')):
    starts = np.arange(startPoint,endPoint,step=step)
    ends = starts + np.timedelta64(1, 'D')
    return starts,ends

def fitGaussian(data,ax):
    mu, std = norm.fit(data) 
    
    xmin, xmax = ax.get_xlim()
    x = np.linspace(np.floor(xmin), np.ceil(xmax), int(10E4))
    p = norm.pdf(x, mu, std)

    return mu,std,p,x,xmin,xmax

def getFWHM_from_gaussian(sigma):
    return 2*np.sqrt(np.log(2)*2)*sigma

ModuleNotFoundError: No module named 'lsst'

In [1]:
client = makeEfdClient("summit_efd")
topic = "lsst.sal.Scheduler.logevent_largeFileObjectAvailable"
start = Time("2025-07-07T03:27:10.99")
end = Time("2025-07-07T06:27:10.993")
dataDir = "/home/smacbride/Data/"

NameError: name 'makeEfdClient' is not defined

In [76]:
urls = await client.select_time_series(topic,"url",start,end,index=1)

In [77]:
urls

,url
2025-07-07 03:32:07.490991+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2025-07-07 04:15:18.911698+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2025-07-07 05:16:43.085042+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2025-07-07 05:24:19.168340+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2025-07-07 05:30:36.757521+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2025-07-07 05:37:03.931738+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...
2025-07-07 06:10:31.709306+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...


In [78]:
# dest, _ = urllib.request.urlretrieve(url=str(urls["url"].iloc[4]), filename=os.path.join(dataDir,"scheduler_snapshot.p"))

In [2]:
with open("Scheduler_1_Scheduler_1_2025-07-28T23_30_28.099.p", "rb") as fp:
    scheduler, conditions = pickle.load(fp)

AttributeError: Can't get attribute 'AltAzShadowTimeLimitedBasisFunction' on <module 'rubin_scheduler.scheduler.basis_functions.mask_basis_funcs' from '/opt/lsst/software/stack/conda/envs/lsst-scipipe-10.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/basis_functions/mask_basis_funcs.py'>

In [80]:
import rubin_scheduler.scheduler.basis_functions.mask_basis_funcs as maskBasisFuncs

In [82]:
maskBasisFuncs.AltAzShadowMaskBasisFunction

rubin_scheduler.scheduler.basis_functions.mask_basis_funcs.AltAzShadowMaskBasisFunction